In [38]:
import numpy as np  
import pandas as pd
from sklearn.model_selection import train_test_split
from numpy.random import rand

In [140]:
# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
 
def relu(z):
    return np.maximum(0,z)

def linear(z):
    return z

# loss function
def mse(y, y_pred):
    return np.mean((y - y_pred)**2)

def scale(x):
    return (x - x.mean(axis=0)) / x.std(axis=0)

In [174]:
class NeuralNetwork:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def set_architecture(self, input_n, hidden_n, output_n, Eta=0.001,epochs=1001):
        # number of neurons in each layer
        self.input_n = input_n  #0
        self.hidden_n = hidden_n #1
        self.output_n = output_n #2
        self.Eta = Eta
        self.epochs = epochs
        self.losses = []
        self.weights_input_hidden = np.random.rand(self.x.shape[1],self.hidden_n)
        self.weights_hidden_output = np.random.rand(self.hidden_n, self.output_n)
        
#         self.weights_input_hidden = np.random.randn(self.x.shape[1], self.hidden_n) * np.sqrt(1 / self.x.shape[1])
#         self.weights_hidden_output = np.random.randn(self.hidden_n, self.output_n) * np.sqrt(1 / self.hidden_n)
        self.bias_hidden = np.zeros((1, hidden_n))
        self.bias_output = np.zeros((1, output_n))
        print(f"Initialized W1: {self.weights_input_hidden}")
        print(f"Initialized W2: {self.weights_hidden_output}")
        print("")
        

    def changeRate(self,x):
        self.Eta=x

    def cost_function(self, y, y_pred):
        return mse(y, y_pred)
    

    def forward(self,x):
    
        self.Z1 = np.dot(x, self.weights_input_hidden) + self.bias_hidden
        self.a1 = sigmoid(self.Z1)

        self.Z2 = np.dot(self.a1, self.weights_hidden_output) + self.bias_output
        self.a2 = linear(self.Z2)
        
        return self.a2

    
        
    def backward(self):
        #the error at the output layer
        delta_output = self.y - self.a2
        
#         delta_output = error_output * sigmoid(self.Z2) * (1 - sigmoid(self.Z2))
        
        # Update weights & bais for output layer
        self.weights_hidden_output += self.Eta * np.dot(self.a1.T, delta_output)
        self.bias_output += self.Eta * delta_output.mean(axis=0, keepdims=True)

        
        #the error at the hidden layer
        error_hidden = np.dot(delta_output, self.weights_hidden_output.T)
        
        delta_hidden = error_hidden *  sigmoid(self.Z1) * (1 - sigmoid(self.Z1))
        
        # Update weights & bais for hidden layer
        self.weights_input_hidden += self.Eta * np.dot(self.x.T, delta_hidden)
        self.bias_hidden += self.Eta * delta_hidden.mean(axis=0, keepdims=True)
        
        
        return self
    
            
    def fit(self):
        for i in range(self.epochs):
            y_pred = self.forward(self.x)
            error = self.cost_function(self.y, y_pred)
            self.losses.append(error)
            self.backward()
            if i % 100 == 0:
                print("Iteration # ", i)
                print("Cost: \n", error, "\n")

In [221]:
# Load data from the Excel file
data = pd.read_excel("concrete_data.xlsx")
x = data.iloc[:, :4].values  # features
y = data.iloc[:, 4].values.reshape(-1, 1)  # target  

# Normalize the data
x = scale(x)
# print(x)

# Split the data into training and testing sets
train_size = int(0.75 * len(data))
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]



In [220]:
NN = NeuralNetwork(x_train, y_train)
NN.set_architecture(x_train.shape[1], 3, 1, Eta = 0.001, epochs = 1001)
NN.fit()
# NN.plot_losses()

# Test the network on test data
predictions = NN.forward(x_test)
error = NN.cost_function(y_test, predictions)
print("\nMSE:", error)

Initialized W1: [[0.12291547 0.47686985 0.96138   ]
 [0.33017209 0.92659191 0.53276169]
 [0.08531362 0.35919437 0.15715652]
 [0.12418095 0.16985684 0.89920868]]
Initialized W2: [[0.06334467]
 [0.25676387]
 [0.04781342]]

Iteration #  0
Cost: 
 2042.8219302299983 

Iteration #  100
Cost: 
 298.73838660034704 

Iteration #  200
Cost: 
 183.98529505641562 

Iteration #  300
Cost: 
 183.27217807458604 

Iteration #  400
Cost: 
 142.23592884095734 

Iteration #  500
Cost: 
 147.99731623763572 

Iteration #  600
Cost: 
 151.20709356486165 

Iteration #  700
Cost: 
 131.44758218107253 

Iteration #  800
Cost: 
 134.45330303876295 

Iteration #  900
Cost: 
 148.12048628084392 

Iteration #  1000
Cost: 
 122.76770481800011 


MSE: 63.93848152381845
